
# How to generate text:デコード方法の違いを利用したテキスト生成

### はじめに

改良されたトランスフォーマアーキテクチャと大規模な教師なし学習データの他に、**より良いデコーディング方法**も重要な役割を果たしています。

さまざまなデコーディング戦略の概要を簡単に説明し、`transformers` ライブラリを使って、わずかな労力でそれらを実装する方法を紹介します。

以下の機能はすべて 自己回帰モデルのテキスト生成に使用することができます。自己回帰モデルでは、単語列の確率分布が次の単語の条件付き分布の積に分解できるという仮定に基づいています。

$$ P(w_{1:T} | W_0 ) = \prod_{t=1}^T P(w_{t} | w_{1: t-1}, W_0) \text{ ,with }  w_{1: 0} = \emptyset, $$

$W_0$ は初期の *文脈* の単語列です。単語列の長さ $T$ は通常 *on-the-fly* で決定され、タイムステップ $t=T$ に対応します。

ここでは、主に *Greedy search*, *Beam search*, *Top-K sampling*, *Top-p sampling* を中心に、有名なデコード方法を紹介します。


準備
1.  ライブラリのインストール
2.  モデルのロード

今回は、rinna/japanese-gpt2-smallを使用します。




In [ ]:
!pip install sentencepiece
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 45.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 71.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 24.0 MB/s eta 0:00:00


In [1]:
from transformers import T5Tokenizer, AutoModelForCausalLM, GenerationConfig

# 0. モデルとトークナイザーの定義
tokenizer = T5Tokenizer.from_pretrained("rinna/japanese-gpt2-small")
tokenizer.do_lower_case = True  # rinna/japanese-gpt2特有のハック
model = AutoModelForCausalLM.from_pretrained(
    "rinna/japanese-gpt2-small",
    pad_token_id=tokenizer.eos_token_id  # warningを避けるために、padにEOSトークンを割りあてる
)

### **Greedy Search**
Greedy Searchでは、単に次の単語として最も確率の高い単語を選択するだけです：$w_t = argmax_{w}P(w | w_{1:t-1})$ を各タイムステップ $t$. 次のスケッチは、Greedy Searchを示しています。

![Greedy Search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/greedy_search.png)

単語 $\text{"The"}$から始めると アルゴリズムは 
貪欲(Greedy)に確率の高い次の単語を選んでいくので、最終的に生成される単語列は、$\text{"The", "nice", "woman"}$で、全体の確率は$0.5 \times 0.4 = 0.2$となります。

日本語で試してみましょう。



In [3]:
text = "福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。どんな観光体験が福岡に必要でしょうか"

inputs = tokenizer(text, add_special_tokens=False, return_tensors="pt")["input_ids"]

# テキスト生成用のconfigクラス
config = GenerationConfig(
    max_new_tokens=250,  # 出力の長さが250になるまで生成を続ける
)
outputs = model.generate(inputs, generation_config=config)
generated = tokenizer.decode(outputs[0])

# warinigが出ると見にくいので、出力の開始位置を表示
print("Output:\n" + 100 * '-')
# 見やすさのために、改行コードを挿入
print("。\n".join(generated.split("。")))

Output:
----------------------------------------------------------------------------------------------------
福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
どんな観光体験が福岡に必要でしょうか。
 福岡の観光スポットとして、まずは博多駅が挙げられます。
博多駅には、博多駅ビルや博多駅ビルの2つの商業施設があります。
 博多駅ビルは、博多駅ビルの2階にある商業施設です。
博多駅ビルは、博多駅ビルの2階にある商業施設です。
 博多駅ビルは、博多駅ビルの2階にある商業施設です。
博多駅ビルは、博多駅ビルの2階にある商業施設です。
 博多駅ビルは、博多駅ビルの2階にある商業施設です。
博多駅ビルは、博多駅ビルの2階にある商業施設です。
 博多駅ビルは、博多駅ビルの2階にある商業施設です。
博多駅ビルは、博多駅ビルの2階にある商業施設です。
 博多駅ビルは、博多駅ビルの2階にある商業施設です。
博多駅ビルは、博多駅ビルの2階にある商業施設です。
 博多駅ビルは、博多駅ビルの2階にある商業施設です。
博多駅ビルは、博多駅ビルの2階にある商業施設です。
 博多駅ビルは、博多駅ビルの2階にある商業施設です。
博多駅ビルは、博多駅ビルの2階にある商業施設です


文脈に沿って生成する方法は合理的ですが、モデルはすぐに繰り返しを始めてしまいます。これは一般的なテキスト生成において非常に一般的な問題であり、Greedy SearchやBeam Searchにおいては特に強いようです。

しかし、Greedy Searchの主な欠点は、上のスケッチで見られるように、低確率の単語の背後に隠された高確率の単語を見逃してしまうことです。

条件付き確率の高い$0.9$の単語$\text{"has"}$は、2番目に高い条件付き確率しかない単語$\text{"dog"}$の後ろに隠れています。

Beam Searchを使えば、この問題を軽減することができます。


### **Beam search**
Beam Searchでは、各ステップで最も可能性の高い仮説の `num_beams` を保持し、最終的に全体的に最も高い確率を持つ仮説を選択することで、隠れた高確率の単語列を見落とすリスクを減らすことができます。ここでは、`num_beams=2`を用いて説明します。

![Beam search](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/beam_search.png)


ステップ$1$では、最も可能性の高い仮説$\text{"The", "woman"}$の他に、2番目に可能性の高い仮説$\text{"The", "dog"}$も追跡しています。ステップ$2$では、$\text{"The", "dog", "has"}$の確率は、$0.2$の$\text{"The", "nice", "woman"}$よりも$0.36$高いことがわかりました。この簡単な例で最も可能性の高い単語列を見つけました。

Beam Searchは、Greedy Searchよりも高い確率で常に出力を見つけますが、最も可能性の高い出力を見つけることは保証されていません。

それでは、`transformers`でBeam Searchをどのように使うことができるか見てみよう。ここでは、`num_beams > 1` と `early_stopping=True` を設定し、すべてのビーム仮説がEOSトークンに到達した時点で生成が終了するようにしています。

In [4]:
# テキスト生成用のconfigクラス
config = GenerationConfig(
    max_new_tokens=250,  # 出力の長さが250になるまで生成を続ける
    num_beams=5,  # Beam Searchの有効化
    early_stopping=True
)

beam_output = model.generate(inputs, generation_config=config)

generated = tokenizer.decode(beam_output[0], skip_special_tokens=True)

# warinigが出ると見にくいので、出力の開始位置を表示
print("Output:\n" + 100 * '-')
# 見やすさのために、改行コードを挿入
print("。\n".join(generated.split("。")))

Output:
----------------------------------------------------------------------------------------------------
福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
どんな観光体験が福岡に必要でしょうか。
 福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
 福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
 福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
 福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
 福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
 福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
 福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
 福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
 福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
 福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
 福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力


若干まともになっているようですが、出力には同じ単語列の繰り返しが含まれています。 
簡単な解決策は、*n-grams* (*a.k.a.* $n$単語の単語列)ペナルティを導入することです。最も一般的な*n-grams*ペナルティは、すでに見た*n-gram*を作る可能性のある次の単語の確率を手動で$0$に設定することで、*n-gram*が2回出現しないようにします。

それでは、`no_repeat_ngram_size=2`を設定して、*2-gram*が繰り返し出現しないようにしてみましょう。

In [5]:
# テキスト生成用のconfigクラス
config = GenerationConfig(
    max_new_tokens=250,  # 出力の長さが250になるまで生成を続ける
    num_beams=5,  # Beam Searchの有効化
    early_stopping=True,
    no_repeat_ngram_size=2,  # 繰り返し出現しないようにオプションを追加
)
beam_output = model.generate(inputs, generation_config=config)

generated = tokenizer.decode(beam_output[0], skip_special_tokens=True)

# warinigが出ると見にくいので、出力の開始位置を表示
print("Output:\n" + 100 * '-')
# 見やすさのために、改行コードを挿入
print("。\n".join(generated.split("。")))

Output:
----------------------------------------------------------------------------------------------------
福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
どんな観光体験が福岡に必要でしょうか? 今回は、福岡でおすすめの観光スポットをご紹介していきます。
 博多駅や天神駅から徒歩5分ほどの場所にある「博多ラーメン」。
博多のラーメンといえば、博多豚骨醤油ラーメンが有名ですよね。
そんな博多で人気のお店をピックアップしてみましたので、ぜひ参考にしてみてくださいね! こちらの店は博多駅から歩いて5分の場所にあり、アクセスも抜群です。
店内はカウンター席とテーブル席があり、おひとり様でも気軽に入れるのも嬉しいポイントですね♪ 店内に入ると、まず目に飛び込んでくるのがラーメンの看板。
ラーメン好きにはたまらないお店だと思います。
 お店に入るとまず目に入るのはラーメンとつけ麺がセットになったお得なセットメニュー。
お一人様はもちろん、家族連れやカップルでのお食事にもぴったりです! お店の雰囲気はとてもアットホームな感じでした。
店員さんも気さくな方ばかりなので、初めての方でも安心してお食事を楽しめますよ。
 また、こちらのお店の名物は「つけめん」!お客さんの好みに合わせて麺を選べるのはもちろんのこと、スープの濃さや麺の太さ


だいぶ良くなりました。繰り返しが出てこなくなったのがわかります。

Beam Searchのもう一つの重要な特徴は、生成後にトップのビームを比較して、目的に合ったビームを選択できることです。

`num_return_sequences` というパラメータにビームを何本返すかを設定します。ただし、`num_return_sequences <= num_beams` となるよう
にしてください。

In [6]:
# テキスト生成用のconfigクラス
config = GenerationConfig(
    max_new_tokens=250,  # 出力の長さが250になるまで生成を続ける
    num_beams=5,  # Beam Searchの有効化
    early_stopping=True,
    no_repeat_ngram_size=2,  # 繰り返し出現しないようにオプションを追加
    num_return_sequences=5,  # 返すセンテンスの数
)

# beam_output => beam_outputsとなっています。
beam_outputs = model.generate(inputs, generation_config=config)

# warinigが出ると見にくいので、出力の開始位置を表示
print("Output:\n" + 100 * '-')
for i, beam_output in enumerate(beam_outputs):
  generated = tokenizer.decode(beam_outputs[0], skip_special_tokens=True)

  print(f"\n{i}番の生成結果")
  # 見やすさのために、改行コードを挿入
  print("。\n".join(generated.split("。")))

Output:
----------------------------------------------------------------------------------------------------

0番の生成結果
福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
どんな観光体験が福岡に必要でしょうか? 今回は、福岡でおすすめの観光スポットをご紹介していきます。
 博多駅や天神駅から徒歩5分ほどの場所にある「博多ラーメン」。
博多のラーメンといえば、博多豚骨醤油ラーメンが有名ですよね。
そんな博多で人気のお店をピックアップしてみましたので、ぜひ参考にしてみてくださいね! こちらの店は博多駅から歩いて5分の場所にあり、アクセスも抜群です。
店内はカウンター席とテーブル席があり、おひとり様でも気軽に入れるのも嬉しいポイントですね♪ 店内に入ると、まず目に飛び込んでくるのがラーメンの看板。
ラーメン好きにはたまらないお店だと思います。
 お店に入るとまず目に入るのはラーメンとつけ麺がセットになったお得なセットメニュー。
お一人様はもちろん、家族連れやカップルでのお食事にもぴったりです! お店の雰囲気はとてもアットホームな感じでした。
店員さんも気さくな方ばかりなので、初めての方でも安心してお食事を楽しめますよ。
 また、こちらのお店の名物は「つけめん」!お客さんの好みに合わせて麺を選べるのはもちろんのこと、スープの濃さや麺の太さ

1番の生成結果
福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
どんな観光体験が福岡に必要でしょうか? 今回は、福岡でおすすめの観光スポットをご紹介していきます。
 博多駅や天神駅から徒歩5分ほどの場所にある「博多ラーメン」。
博多のラーメンといえば、博多豚骨醤油ラーメンが有名ですよね。
そんな博多で人気のお店をピックアップしてみましたので、ぜひ参考にしてみてくださいね! こちらの店は博多駅から歩いて5分の場所にあり、アクセスも抜群です。
店内はカウンター席とテーブル席があり、おひとり様でも気軽に入れるのも嬉しいポイントですね♪ 店内に入ると、まず目に飛び込んでくるのがラーメンの看板。
ラーメン好きにはたまらないお店だと思います。
 お店に入るとまず目

見ての通り、5つのBeam Searchのデコード結果は互いにわずかに異なるだけです。

Beam Searcが最良の選択肢ではない理由がいくつか提唱されています。

- Beam Searcは、機械翻訳や要約のように、目的とする生成の長さがある程度予測可能なタスクでは非常にうまく機能します。しかし、これは、対話やストーリーの生成など、想定する出力長が大きく変化する可能性がある場合はそうではありません。

- Beam Searcが繰り返し生成に大きく苦しむことを見てきました。これは、テキスト生成における*n-gram*やその他のペナルティで制御することが特に困難です。なぜなら、強制的に「繰り返しなし」にすることと、同一の*n-gram*の繰り返しサイクルとの間の良いトレードオフを見つけることは、多くの微調整を必要とするからです。

- [Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751)で論じられているように、高品質の人間のテキストは、高確率に従わない。言い換えれば、人間は、生成されたテキストには驚きを与えているし、つまらない/予測可能なものをわざわざ書かないということです。著者らは、モデルが人間のテキストに与えるであろう確率と、ビームサーチが行うこととの比較をプロットすることで、このことをうまく示しています。

![alt text](https://blog.fastforwardlabs.com/images/2019/05/Screen_Shot_2019_05_08_at_3_06_36_PM-1557342561886.png)


人間は0から文を考えることができます。ここで、ランダム性を導入してみましょう。

### **Sampling**

その最も基本的な形式であるサンプリングは、その条件付き確率分布に従って、次の単語 $w_t$ をランダムに選ぶことを意味します。

$w_t \sim P(w|w_{1:t-1})$.

上記を例に、サンプリング時のテキスト生成を可視化したものが以下の図です。


![vanilla_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/sampling_search.png)

サンプリングを使ったテキスト生成は、もはや*deterministic*ではないことが明らかになる。単語$\text{"car"}$は 条件付き確率 $P(w | \text{"The"})$からサンプリングされます。続いて $\text{"drives"}$は $P(w | \text{"The"}, \text{"car"})$からサンプリングされます。

`transformers`では、`do_sample=True`とし、`top_k=0` で*Top-K*のサンプリングを無効にします(これについては後述)。以下では、説明のために `random_seed=0` を固定します。random_seedは自由に変更してください。


In [7]:
from transformers import set_seed

# 結果の再現性担保のための設定。値を変えてみると、結果が変わります。
set_seed(21)

# テキスト生成用のconfigクラス
config = GenerationConfig(
    max_new_tokens=250,  # 出力の長さが250になるまで生成を続ける
    do_sample=True,  # サンプリングを有効化
    top_k=0   #（k=0の場合は、Top-Kサンプリングが無効）
)

sample_output = model.generate(inputs, generation_config=config)

generated = tokenizer.decode(sample_output[0], skip_special_tokens=True)

# warinigが出ると見にくいので、出力の開始位置を表示
print("Output:\n" + 100 * '-')
# 見やすさのために、改行コードを挿入
print("。\n".join(generated.split("。")))

Output:
----------------------------------------------------------------------------------------------------
福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
どんな観光体験が福岡に必要でしょうか。
街の歴史を感じることが出来るのではありませんか。
福岡の落ち着いた雰囲気でお団子に生まれ変わらせてくれるお店を紹介してまいります。
ちなみに司馬遼太郎の故郷である福岡を総なめするため、この場所にお団子が作られたのです。
博多にあるお団子店はその多くが江戸時代に「高橋(斉彬)」が作成したお菓子の味が本場中国製法なので『アジアの道玄坂』と呼ばれているそうです。
近年では大阪など大都市だけではなく日常の様子も再現するお洒落な店舗も入店しているそうです。
別途事前に調べる必要がありますが、詳しく教えてくれるお店もあり詳しく地域ごとに紹介してくださいました。
相談にのってくれるお店もありますので初めてお団子の通販でお団子を購入予定の方はオーダーしてからのお団子の購入をしたいなら、こちらのページを参考にしてくださいね。
 福岡発祥のお団子屋さんを紹介しているウェブサイトです。
カテゴリーにお店のなかったお店を紹介していただき、そのページがあります。



面白いですね。テキストは大丈夫そうですが、よく見るとあまりまとまりがありません。3-grams *お団子に生まれ* と *店舗も入店* は非常に奇妙で、人間が書いたようにはみえません。これが単語列をサンプリングするときの大きな問題です。モデルはしばしば支離滅裂な失言を生成します。

トリックは、[softmax](https://en.wikipedia.org/wiki/Softmax_function#Smooth_arg_max)の`temperature`を下げることで、分布 $P(w|w_{1:t-1})$ をシャープにすることです(高確率の単語のより確率を上げ、低確率の単語の確率を下げる)。

上の例で`temperature`を適用した例は以下のようになります。

![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/sampling_search_with_temp.png?raw=true)

ステップ $t=1$ の次の単語の条件付き分布はかなりシャープになり、 $\text{"car"}$ が選択される可能性はほとんどなくなります。


`temperature=0.7` を設定して、分布をシャープにさせる方法を見てみましょう。

In [8]:
# 結果の再現性担保のための設定。値を変えてみると、結果が変わります。
set_seed(21)

# テキスト生成用のconfigクラス
config = GenerationConfig(
    max_new_tokens=250,  # 出力の長さが250になるまで生成を続ける
    do_sample=True,  # サンプリングを有効化
    top_k=0,   #（k=0の場合は、Top-Kサンプリングが無効）
    temperature=0.7  # 低確率の候補に対する感度を下げる
)

sample_output = model.generate(inputs, generation_config=config)

generated = tokenizer.decode(sample_output[0], skip_special_tokens=True)

# warinigが出ると見にくいので、出力の開始位置を表示
print("Output:\n" + 100 * '-')
# 見やすさのために、改行コードを挿入
print("。\n".join(generated.split("。")))

Output:
----------------------------------------------------------------------------------------------------
福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
どんな観光体験が福岡に必要でしょうか。
 観光スポットとしては、福岡の商業施設やデパート、官公庁などが有名です。
博多駅周辺は、博多駅や天神駅へも、飛行機を降りて徒歩5分以内で行くことができます。
観光スポットとしてはあまり有名なスポットではないのですが、博多駅周辺は一日で回れる距離で、ホテルや旅館が立ち並ぶエリアです。
飲食店も多く、夜遅くまで営業しているので、夜遅くまで営業しているお店は便利です。
 観光客には、博多駅周辺や天神駅周辺で有名な飲食店が好まれます。
飲食店では、別途に様々なメニューがあり、お土産選びの参考になります。
 地域ごとに特色のある飲食店も多いのですが、特に有名なお店は、博多駅周辺や天神駅周辺に立地しています。
有名なお店は、福岡で有名なラーメン屋や焼き鳥屋、居酒屋などです。
 福岡のグルメスポットとして、観光客には、博多駅周辺が特に人気です。
観光客にお土産選びの参考になれば嬉しいですね。
 レジャー施設やスポーツ施設、観光地もたくさんあるので、福岡に観光に行くなら、観光スポットを観光しに行きましょう。
 博多駅周辺は、観光地として有名なので、有名な飲食店がたくさんあります。



話題も変わってしまいましたが、よくなったように見えます。変なn-gramが少なくなって、出力がもう少しまとまりが出てきました。`temperature`を適用することで分布のランダム性を低くすることができますが、その限界では、`temperature` $ \to 0$ を設定すると、`temperature`スケーリングされたサンプリングはGreedy Searchと同じになり、以前と同じ問題に悩まされることになります。




### **Top-K Sampling**
**Top-K**サンプリングと呼ばれる単純だが非常に強力なサンプリングスキームがあります。*Top-K* サンプリングでは、最も可能性の高い*K*次の単語をフィルタリングし、それらの*K*次の単語のみに確率を再分配します。
GPT2では、このサンプリング方式を採用しており、これがテキスト生成に成功した理由の一つと言われています。

上の例の両方のサンプリングステップで使用される単語の範囲を3単語から10単語に拡張して、*Top-K*サンプリングをより良く説明しています。

![top_k_sampling](https://raw.githubusercontent.com/patrickvonplaten/scientific_images/master/top_k_sampling.png)

$k = 6$としたので、両方のサンプリングステップでは、サンプリングプールを6単語に制限します。$V_{\text{top-K}}$と定義された6つの最も可能性の高い単語は、最初のステップでは全体の確率の3分の2しか含まれていませんが、2つ目のステップではほぼすべての確率を含んでいます。それにも関わらず、変な候補 $\text{"not", "the", "small", "telled"}$ を除去することに成功していることがわかります。


`top_k=50`を設定して、*Top-K*がどのように使えるか見てみましょう。

In [14]:
# 結果の再現性担保のための設定。値を変えてみると、結果が変わります。
set_seed(4)

# テキスト生成用のconfigクラス
config = GenerationConfig(
    max_new_tokens=250,  # 出力の長さが250になるまで生成を続ける
    do_sample=True,  # サンプリングを有効化
    top_k=50,  # 確率の高い上位k個に制限する（k=0の場合は、Top-Kサンプリングが無効）
)
sample_output = model.generate(inputs, generation_config=config)

generated = tokenizer.decode(sample_output[0], skip_special_tokens=True)

# warinigが出ると見にくいので、出力の開始位置を表示
print("Output:\n" + 100 * '-')
# 見やすさのために、改行コードを挿入
print("。\n".join(generated.split("。")))

Output:
----------------------------------------------------------------------------------------------------
福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
どんな観光体験が福岡に必要でしょうか。
これから、楽しみにしていきたいと思います。
 観光で訪れたことがない人も意外なほど、多くの魅力があります。
福岡を訪れたときは、観光で訪れたときに地元の方に話しかけられたり、話相手になっていただければ嬉しいですね。
 街は博多を代表して、福岡市を中心とした市街地に多く存在していますが、特に観光では、福岡市を中心とした博多を中心に、福岡市を中心とした中心市街地に点在しています。
 観光で訪れる際には、多くの方にその魅力を知ってもらえると嬉しいですね。
 街の中にはカフェやレストラン、宿泊施設などがあり、ゆっくりと過ごしているだけで雰囲気が伝わってきますが、福岡には「ランチスポット」がいくつもあり、グルメとショッピングを楽しむための拠点にもなります。
 グルメやショッピングに便利な飲食店は、福岡だけではなく日本各地にあるため、特に観光の際には観光を充実させたいところになります。
福岡では、福岡のグルメに関するお店を簡単に選ぶことができますし、グルメに関しても福岡のグルメに詳しい人におすすめのお店を簡単に選ぶことができます。
 グルメを楽しみたい人はたくさんいますし、お店はたくさんありますが、その中でもおすすめのスポットを見ていきましょう。
 福岡にあるお店を幾つか紹介していきますので、参考にしてみてください。



全然悪くないですよね!? このテキストは、間違いなく、これまでのところ最も*人間らしい* テキストです。
*Top-K*サンプリングで懸念されることは、次の単語の確率分布 $P(w|w_{1:t-1})$からフィルタリングされる単語の数を動的に適応させないことです。
これは問題となります。ある単語は非常に鋭い分布（上のグラフの右側の分布）からサンプリングされ、他の単語はより平坦な分布（上のグラフの左側の分布）からサンプリングされるかもしれないからです。

ステップ $t=1$ では、*Top-K* は、次のような可能性を排除します。
サンプル $\text{"people", "big", "house", "cat"}$ が妥当な候補と思われます。一方で、ステップ$t=2$では、この方法では、$\text{"down", "a"}$のような、かなり合わない単語が単語のサンプルプールに含まれています。このように、サンプルプールを固定サイズ*K*に制限することは、鋭い分布のためにモデルが失言を生成する危険にさらし、平坦な分布のためにモデルの創造性を制限する可能性があります。
この直感は、[Ari Holtzman et al. (2019)](https://arxiv.org/abs/1904.09751)が**Top-p**-または**nucleus** - samplingを作成することにつながりました。


### **Top-p (nucleus) sampling**

*Top-p*サンプリングでは、最も可能性の高い*K*語のみからサンプリングするのではなく、累積確率が確率*p*を超える可能性のある最小の単語集合から選択します。確率は、この単語集合の中で再分配されます。このようにして、単語集合のサイズ（*a.k.a.* 集合内の単語数）は、次の単語の確率分布に応じて動的に増減することができます。


![top_p_sampling](https://github.com/patrickvonplaten/scientific_images/blob/master/top_p_sampling.png?raw=true)

$p=0.92$とすると、*Top-p*サンプリングでは、$V_{\text{top-p}}$と定義されている確率の$p=92\%$を超える単語の*最小*数を選びます。最初の例では、最も可能性の高い9つの単語が含まれていましたが、2番目の例では、92％を超えるためには、上位3つの単語を選ぶだけです。とてもシンプルです。これは、次の単語が予測しにくい単語の広い範囲を保持していることがわかります*e.g.* $P(w | {\text{"The"}})$と、次の単語が予測しやすいと思われるいくつかの単語だけを保持していることがわかります*e.g.* $P(w | {\text{"The", "car"}})$.


`0 < top_p < 1`を設定して、*Top-p*サンプリングを有効にします。

In [15]:
# 結果の再現性担保のための設定。値を変えてみると、結果が変わります。
set_seed(4)

# テキスト生成用のconfigクラス
config = GenerationConfig(
    max_new_tokens=250,  # 出力の長さが250になるまで生成を続ける
    do_sample=True,  # サンプリングを有効化
    top_p=0.92,  # 92%の確率の単語のみからサンプリングする。
    top_k=0,  # top_kサンプリングを無効化
)
sample_output = model.generate(inputs, generation_config=config)

generated = tokenizer.decode(sample_output[0], skip_special_tokens=True)

# warinigが出ると見にくいので、出力の開始位置を表示
print("Output:\n" + 100 * '-')
# 見やすさのために、改行コードを挿入
print("。\n".join(generated.split("。")))

Output:
----------------------------------------------------------------------------------------------------
福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
どんな観光体験が福岡に必要でしょうか。
これから、楽しみにしていきたいと思います。
 観光で訪れたことがない人も意外と多いと思うので、この記事では福岡の観光施設や駅周辺、駐車場の場所について紹介していきたいと思います。
福岡の名所の一つである観光地の一つである天神を周り、一日で効率よく周ることができますので、福岡の代表的な観光スポットの一つとして定着している方もいると思います。
 また、地元の皆さんはちょっと足をのばすだけでもお店やお店がありますが、どんなお店が入っているのでしょうか。
こちらの記事で、現在あるお店を紹介していきたいと思います。
観光スポットの一つの、天神の立ち食いそば「オムそば」をご紹介します。
 「オムそば」とは、奥行きのある赤いうどんやわらび、鍋の具材などから、見た目は地うどんのようですが、味はあっさりしていて、芋の風味と透明感のあるトリュフのような風味があります。
 駅から歩いても近く、天神に観光に訪れた人も、一人でも気軽に立ち寄れるので、人気があります。
 というのも、初めては電車に乗ることが多いと思いますが、jr駅周辺など電車でのアクセスはとても便利で、福岡を観光したのに駅から直接駅まで行って食事ができる


いい感じですね、人間が書いた感じがします。まあでもまだまだですね。

理論的には *Top-p* の方が *Top-K* よりもエレガントに見えますが、実際にはどちらの方法もうまく機能します。*Top-p*は*Top-K*と組み合わせて使用することもでき、動的な選択を可能にしながら、非常に低いランクの単語を避けることができます。

最後に、独立してサンプリングされた複数の出力を得るために、パラメータ `num_return_sequences > 1` を設定することができます。

In [22]:
# 結果の再現性担保のための設定。値を変えてみてると、結果が変わります。
set_seed(5)

# テキスト生成用のconfigクラス
config = GenerationConfig(
    max_new_tokens=250,  # 出力の長さが250になるまで生成を続ける
    do_sample=True,  # サンプリングを有効化
    top_p=0.95,  # 95%の確率の単語のみからサンプリングする。
    top_k=50,  # top_kとtop_pは同時に使える
    num_return_sequences=3  # 候補を３つ返す
)
sample_outputs = model.generate(inputs, generation_config=config)

# warinigが出ると見にくいので、出力の開始位置を表示
print("Output:\n" + 100 * '-')
for i, sample_output in enumerate(sample_outputs):
  generated = tokenizer.decode(sample_output, skip_special_tokens=True)

  print(f"\n{i}番の生成結果")
  # 見やすさのために、改行コードを挿入
  print("。\n".join(generated.split("。")))

Output:
----------------------------------------------------------------------------------------------------

0番の生成結果
福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
どんな観光体験が福岡に必要でしょうか。
 九州と聞くと博多のイメージが浮かびますが、福岡も博多もそれぞれに魅力があります。
なぜなら、福岡は九州の真ん中から近くて、福岡の市街地は九州の中でもかなり都会です。
福岡は、九州の中では比較的温暖な土地にあります。
気温が10度以下になると風が吹いていて、暑すぎるということもありません。
また、福岡は冬も冬も寒暖の差が激しいので、過ごしやすいです。
観光としてはあまり魅力がありませんが、自然を満喫したいという方は、自然体験をおすすめします。
 福岡の定番土産といえば、博多味噌が有名です。
博多は博多港に面していて、博多味噌は博多から海産物を買うこともできます。
博多味噌は海産物として有名な博多味噌が有名です。
福岡が一番好きな場所という方も多いですよね。
 福岡土産として人気があるのは、福岡の美味しいものがたくさん販売されているお店や土産店です。
福岡県の福岡は、美味しい食べ物やおいしいものがいっぱいありますので、お土産探しを楽しんでみてはいかがでしょうか。
福岡には、たくさんの観光スポットがあります。
 福岡といえば、やっぱりおしゃれで流行も

1番の生成結果
福岡のご飯が美味しいと評判ですが、実際に観光地としてどういった魅力があるでしょうか。
どんな観光体験が福岡に必要でしょうか。
 福岡旅行は、福岡空港の博多口から船で博多港に直行し、博多へと向かいます。
空港から船で福岡へと向かい、そこからフェリーで博多駅まで移動します。
フェリーに乗るための運賃は2日間分となりますが、当日の出発や便数は変わります。
 福岡には、国内有数の観光スポットがたくさんありますが、その中でも、特に人気の観光スポットの1つが福岡城です。
 福岡城といえば、日本三大城のひとつで、有名な観光スポットの一つです。
この記事では、福岡城の観光スポットや歴史をご紹介します。
 九州旅行の定番の観光地である、九州五城のひとつ、博多城ですが、

### **その他のパラメタ**
`generate` メソッドには、上では触れられていない追加のパラメータがいくつかあります。ここでは簡単に説明します

- `min_length`は、`min_length`に到達する前にモデルがEOSトークンを生成しないように強制するために使われます。これは要約の際によく使われて、ユーザがより長い出力をしたい場合には一般的に便利です。
- `repetition_penalty` は、既に生成された単語や文脈に属する単語にペナルティを与えるために用いることができます。繰り返しの防止にはかなり効果的ですが、異なるモデルやユースケースには非常に敏感に反応するようです。

- `attention_mask` は、パディングされたトークンをマスクするために使うことができます。
- `pad_token_id`, `bos_token_id`, `eos_token_id`. モデルがこれらのトークンをデフォルトで持っていない場合は、ユーザーが手動で他のトークンIDを選択して表現することができます。

詳細は `generate` 関数 [docstring](https://huggingface.co/transformers/main_classes/model.html?highlight=generate#transformers.TFPreTrainedModel.generate) を参照してください。